<a href="https://colab.research.google.com/github/Durixas/Parkinson-s-Disease-Prediction-Code-and-Data-Repository/blob/main/post_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
roc_auc_score, roc_curve, confusion_matrix, cohen_kappa_score, confusion_matrix, matthews_corrcoef

from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier

import warnings

warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_id = '1irvCs70iiI8WkXSZMMStmfPGfke5ivrP'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data.csv')

data = pd.read_csv('data.csv', header=1) # pd.read_excel() for .xlsx file

# Data Preprocessing

In [ ]:
data

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,250,0,0.80903,0.56355,0.28385,417,416,0.004627,0.000052,0.00064,...,3.0706,3.0190,3.1212,2.4921,3.5844,3.5400,3.3805,3.2003,6.8671,0
752,250,0,0.16084,0.56499,0.59194,415,413,0.004550,0.000220,0.00143,...,1.9704,1.7451,1.8277,2.4976,5.2981,4.2616,6.3042,10.9058,28.4170,0
753,251,0,0.88389,0.72335,0.46815,381,380,0.005069,0.000103,0.00076,...,51.5607,44.4641,26.1586,6.3076,2.8601,2.5361,3.5377,3.3545,5.0424,0
754,251,0,0.83782,0.74890,0.49823,340,339,0.005679,0.000055,0.00092,...,19.1607,12.8312,8.9434,2.2044,1.9496,1.9664,2.6801,2.8332,3.7131,0


In [ ]:
unique_ids = data.groupby('id')['class'].max().reset_index()
id_X = unique_ids['id']
id_y = unique_ids['class']

def split_data_by_id(data, test_size=0.3, random_state=None):
    train_ids, test_ids, _, _ = train_test_split(id_X, id_y, test_size=test_size, stratify=id_y, random_state=random_state)
    train_data = data[data['id'].isin(train_ids)]
    test_data = data[data['id'].isin(test_ids)]
    return train_data, test_data

train_data, test_data, groups = [0]*10, [0]*10, [0]*10
X_train, y_train, X_test, y_test = [0]*10, [0]*10, [0]*10, [0]*10

for i in range(10):
    train_data[i], test_data[i] = split_data_by_id(data, test_size=0.3, random_state=i+1)
    groups[i] = train_data[i].id
    X_train[i], y_train[i] = train_data[i].drop(columns=['id']).iloc[:,:-1], train_data[i].iloc[:,-1]
    X_test[i], y_test[i] = test_data[i].drop(columns=['id']).iloc[:,:-1], test_data[i].iloc[:,-1]

In [ ]:
X_train_scaled, X_test_scaled = [0]*10, [0]*10
for i in range(10):
    scaler = StandardScaler()
    X_train_scaled[i] = pd.DataFrame(
        scaler.fit_transform(X_train[i]),
        index=X_train[i].index,
        columns=X_train[i].columns
    )

    X_test_scaled[i] = pd.DataFrame(
        scaler.transform(X_test[i]),
        index=X_test[i].index,
        columns=X_test[i].columns
    )
    X_train[i], X_test[i] = X_train_scaled[i], X_test_scaled[i]
    train_data[i].iloc[:, 1:-1] = X_train[i]

In [ ]:
def bos(train_data, random_state=42):
    np.random.seed(random_state)

    synthetic_data = train_data.copy()
    grouped = synthetic_data.groupby('id')
    centers = grouped.mean().iloc[:, :-1]
    classes = grouped.first()['class']
    minority_class_ids = centers[classes == 0].index
    majority_class_ids = centers[classes == 1].index


    minority_centers = centers.loc[minority_class_ids]
    distances = cdist(minority_centers.values, minority_centers.values, metric='euclidean')
    np.fill_diagonal(distances, np.inf)

    K = 2
    while len(minority_class_ids) <= len(majority_class_ids) - K:
        selected_idx = np.random.choice(minority_centers.index)
        selected_idx_row = minority_centers.index.get_loc(selected_idx)
        neighbor_indices = distances[selected_idx_row].argsort()[:K]
        neighbor_ids = minority_centers.index[neighbor_indices]

        for neighbor_id in neighbor_ids:
            original_group = grouped.get_group(selected_idx)
            neighbor_group = grouped.get_group(neighbor_id)

            new_id = data.id.max() + len(synthetic_data["id"].unique()) + 1
            for i in range(3):
                original_sample = original_group.sample(n=1, random_state=random_state + i).iloc[:, 1:-1].values
                neighbor_sample = neighbor_group.sample(n=1, random_state=random_state + i).iloc[:, 1:-1].values

                z = np.random.uniform(0, 1)
                new_sample_features = original_sample + z * (neighbor_sample - original_sample)
                new_sample = new_sample_features.squeeze().tolist() + [0]

                new_sample = pd.DataFrame([new_sample], columns=synthetic_data.columns[1:])
                new_sample['id'] = new_id

                synthetic_data = pd.concat([synthetic_data, new_sample], ignore_index=True)

        minority_class_ids = synthetic_data[synthetic_data['class'] == 0]['id'].unique()
    return synthetic_data

for i in range(10):
    synthetic_data = bos(train_data[i])
    X_train[i], y_train[i] = synthetic_data.drop(columns=['id']).iloc[:,:-1], synthetic_data.iloc[:,-1]
    X_train[i] = X_train[i].to_numpy()
    X_test[i] = X_test[i].to_numpy()
    groups[i] = synthetic_data.id

# Model

In [ ]:
def print_grid_search(gs):
    print ("Best score: " + str(gs.best_score_))
    print ("Best parameters set:")
    best_parameters = gs.best_params_
    for param_name in sorted(best_parameters.keys()):
        print(param_name + ':' + str(best_parameters[param_name]))

def get_metrics3(i, model, mode = 'test'):
    if mode == 'train':
        X, y = X_train[i], y_train[i]
    elif mode == 'test':
        X, y = X_test[i], y_test[i]
    results = {}

    # choose the method for post-aggregation

    y_pred_prob = model.predict_proba(X)[:, 1].reshape(-1, 3).mean(axis=1)
    # y_pred_prob = model.predict_proba(X)[:, 1].reshape(-1, 3).min(axis=1)
    # y_pred_prob = model.predict_proba(X)[:, 1].reshape(-1, 3).max(axis=1)

    y_pred = np.round(y_pred_prob).astype(int)
    y = stats.mode(y.to_numpy().reshape(-1, 3), axis=1)[0]

    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    specificity = tn / (tn + fp)

    results['Accuracy'] = accuracy_score(y, y_pred)
    results['Precision'] = precision_score(y, y_pred)
    results['Recall'] = recall_score(y, y_pred)
    results['Specificity'] = specificity
    results['F1'] = f1_score(y, y_pred)
    # results['kappa'] = cohen_kappa_score(y, y_pred)
    results['AUC'] = roc_auc_score(y, y_pred_prob)
    results['MCC'] = matthews_corrcoef(y, y_pred)
    return results

def grid_search(i, model, params):
    group_kfold = GroupKFold(n_splits=5)
    grid = GridSearchCV(model, params, cv=group_kfold)
    grid.fit(X_train[i], y_train[i], groups=groups[i])
    best = grid.best_estimator_
    return get_metrics3(i, best)

def grid_search_print(i, model, params):
    group_kfold = GroupKFold(n_splits=5)
    grid = GridSearchCV(model, params, cv=group_kfold)
    grid.fit(X_train[i], y_train[i], groups=groups[i])
    best = grid.best_estimator_
    print_grid_search(grid)
    return get_metrics3(i, best)

In [ ]:
summary = dict()

## Logistic Regression

In [ ]:
params = {
    'C': [0.01, 0.1, 1.0],
    'penalty': ['l1', 'l2']
}

results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    metrics = grid_search(i, LogisticRegression(solver='liblinear', random_state=42), params)
    for key, value in metrics.items():
        results[key].append(value)

results = dict(results)
summary['Logistic Regression'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['Logistic Regression']).mean()

,0
Accuracy,0.823684
Precision,0.888402
Recall,0.877193
Specificity,0.663158
F1,0.881275
AUC,0.866205
MCC,0.541596


## Decision Tree

In [ ]:
params = {
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    metrics = grid_search(i, DecisionTreeClassifier(random_state=42), params)
    for key, value in metrics.items():
        results[key].append(value)

results = dict(results)
summary['Decision Tree'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['Decision Tree']).mean()

,0
Accuracy,0.772368
Precision,0.855334
Recall,0.840351
Specificity,0.568421
F1,0.845166
AUC,0.752447
MCC,0.414721


## Random Forest

In [ ]:
params = {
    'n_estimators': [100, 200, 300],
    'max_features': ['log2', 'sqrt', 1]
}

results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    metrics = grid_search(i, RandomForestClassifier(random_state=42), params)
    for key, value in metrics.items():
        results[key].append(value)

results = dict(results)
summary['Random Forest'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['Random Forest']).mean()

,0
Accuracy,0.835526
Precision,0.868246
Recall,0.924561
Specificity,0.568421
F1,0.893657
AUC,0.877978
MCC,0.545343


## Gradient Boosting

In [ ]:
params = {
    'n_estimators': [800],
    'learning_rate': [0.01, 0.02],
    'max_depth': [3, 5],
    'max_features': ['sqrt', 'log2']
}

results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    metrics = grid_search(i, GradientBoostingClassifier(random_state=42), params)
    for key, value in metrics.items():
        results[key].append(value)

results = dict(results)
summary['GBDT'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['GBDT']).mean()

,0
Accuracy,0.859211
Precision,0.878299
Recall,0.945614
Specificity,0.600000
F1,0.909744
AUC,0.898523
MCC,0.606156


## XGBoost

In [ ]:
results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    xgb_clf = xgb.XGBClassifier(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=3,
    min_child_weight=1,
    gamma=0.1,
    subsample=0.9,
    colsample_bytree=0.3,
    random_state=42)
    xgb_clf.fit(X_train[i], y_train[i])
    result = get_metrics3(i, xgb_clf)
    for key, value in result.items():
        results[key].append(value)

results = dict(results)
summary['XGBoost'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['XGBoost']).mean()

,0
Accuracy,0.880263
Precision,0.904015
Recall,0.942105
Specificity,0.694737
F1,0.921968
AUC,0.906741
MCC,0.671566


## LightGBM

In [ ]:
params = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.2, 0.3, 0.4],
    'max_depth': [3],
    'num_leaves': [10],
    'subsample': [0.3],
    'colsample_bytree': [0.8, 0.9]
}

results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    metrics = grid_search(i, lgb.LGBMClassifier(n_jobs=-1, verbose=-1), params)
    for key, value in metrics.items():
        results[key].append(value)

results = dict(results)
summary['LightGBM'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['LightGBM']).mean()

,0
Accuracy,0.871053
Precision,0.890214
Recall,0.945614
Specificity,0.647368
F1,0.916630
AUC,0.909141
MCC,0.641949


## KNN

In [ ]:
params = {
    'n_neighbors': [2, 3, 4],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    metrics = grid_search(i, KNeighborsClassifier(n_jobs=-1, algorithm='ball_tree'), params)
    for key, value in metrics.items():
        results[key].append(value)

results = dict(results)
summary['KNN'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['KNN']).mean()

,0
Accuracy,0.748684
Precision,0.891970
Recall,0.757895
Specificity,0.721053
F1,0.817459
AUC,0.791874
MCC,0.435955


## SVM

In [ ]:
params = {
    'C': [0.5, 1, 2],
    'gamma': ['scale', 'auto'],
}

results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    metrics = grid_search(i, SVC(random_state=42, probability=True), params)
    for key, value in metrics.items():
        results[key].append(value)

results = dict(results)
summary['SVM'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['SVM']).mean()

,0
Accuracy,0.855263
Precision,0.861187
Recall,0.964912
Specificity,0.526316
F1,0.909309
AUC,0.854478
MCC,0.585632


## Naive Bayes

In [ ]:
results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    nb = GaussianNB()
    nb.fit(X_train[i], y_train[i])
    result = get_metrics3(i, nb)
    for key, value in result.items():
        results[key].append(value)

results = dict(results)
summary['Naive Bayes'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['Naive Bayes']).mean()

,0
Accuracy,0.732895
Precision,0.895317
Recall,0.731579
Specificity,0.736842
F1,0.803163
AUC,0.811173
MCC,0.420202


## AdaBoost

In [ ]:
results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    ada = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100,
    learning_rate=0.5,
    random_state=42)
    ada.fit(X_train[i], y_train[i])
    result = get_metrics3(i, ada)
    for key, value in result.items():
        results[key].append(value)

results = dict(results)
summary['AdaBoost'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['AdaBoost']).mean()

,0
Accuracy,0.843421
Precision,0.910009
Recall,0.880702
Specificity,0.731579
F1,0.893632
AUC,0.897969
MCC,0.600500


## MLP

In [ ]:
results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    mlp = MLPClassifier(random_state=42, hidden_layer_sizes=(512,256,128,64), learning_rate_init=0.001)
    mlp.fit(X_train[i], y_train[i])
    result = get_metrics3(i, mlp)
    for key, value in result.items():
        results[key].append(value)

results = dict(results)
summary['MLP'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['MLP']).mean()

,0
Accuracy,0.865789
Precision,0.887476
Recall,0.942105
Specificity,0.636842
F1,0.913378
AUC,0.865928
MCC,0.625211


## Stacking

In [ ]:
params = {
    'knn__n_neighbors': [2, 3],
    'svc__C': [1, 10],
    'final_estimator__C': [1, 10],
    'final_estimator__penalty': ['l1', 'l2']
}

results = defaultdict(list)

for i in range(10):
    print(f"\rProcessing iteration {i+1}/10", end='', flush=True)
    estimators = [
    ('knn', KNeighborsClassifier(n_jobs=-1, algorithm='ball_tree')),
    ('svc', SVC(probability=True, gamma='auto', random_state=42))
    ]
    final_estimator = LogisticRegression(solver='liblinear', random_state=42)
    stacking_clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator)

    metrics = grid_search(i, StackingClassifier(estimators=estimators, final_estimator=final_estimator), params)
    for key, value in metrics.items():
        results[key].append(value)

results = dict(results)
summary['Stacking'] = results

Processing iteration 10/10

In [ ]:
pd.DataFrame(summary['Stacking']).mean()

,0
Accuracy,0.853947
Precision,0.856622
Recall,0.968421
Specificity,0.510526
F1,0.908790
AUC,0.862789
MCC,0.578832


# Evaluation

In [ ]:
# import pickle

# with open('summary_mean1.pkl', 'wb') as f:
#     pickle.dump(summary, f)